In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#!conda install -c conda-forge geopy --yes 
#!from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
print('Libraries imported.')

Libraries imported.


In [2]:
#data1 = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',skiprows=1)[0]
#print(data1)

In [3]:
import urllib.request
import bs4 as bs

In [4]:
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'lxml')
table = soup.find('table', attrs={'class':'wikitable sortable'})
table_rows = table.find_all('tr')
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighborhood"])
df.head()

,Postcode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [5]:
# Delete the first row with postcode as 0
df = df.iloc[1:]
df.head()

,Postcode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


In [6]:
df.shape

(288, 3)

# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [7]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df.shape
df = df[~df['Borough'].isin(['Not assigned'])]
df.shape

(211, 3)

In [8]:
df.head()

,Postcode,Borough,Neighborhood
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n
6,M5A,Downtown Toronto,Regent Park\n
7,M6A,North York,Lawrence Heights\n


In [9]:
df.sort_values('Postcode', axis=0, ascending=True, inplace=True)
# remove all '\n' values from the dataframe
df = df.replace('\n','', regex=True)

In [10]:
# rstrip all Neighborhood column
df['Neighborhood'] = df['Neighborhood'].str.strip()
df['Borough'] = df['Borough'].str.strip()

In [11]:
df.head()

,Postcode,Borough,Neighborhood
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern
30,M1C,Scarborough,Port Union
29,M1C,Scarborough,Rouge Hill
28,M1C,Scarborough,Highland Creek


In [12]:
df = df.replace('\n','', regex=True)
df.head()

,Postcode,Borough,Neighborhood
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern
30,M1C,Scarborough,Port Union
29,M1C,Scarborough,Rouge Hill
28,M1C,Scarborough,Highland Creek


# More than one neighborhood can exist in one postal code area

In [13]:
torantodf = pd.DataFrame(columns=['Postcode', 'Borough', 'Neighborhood'])
prev_postcode = ''
prev_neighborhood = ''
prev_borough = ''
unssigned = 'Not assigned'
for index, row in df.iterrows():
    if(row['Postcode'] == prev_postcode):
        prev_neighborhood = prev_neighborhood + ',' + row['Neighborhood']
    else:
        # copy to new df
        torantodf.loc[len(torantodf)] = [prev_postcode,prev_borough ,prev_neighborhood]
        prev_postcode = row['Postcode']
        prev_borough = row['Borough']
        prev_neighborhood = row['Neighborhood']
        
torantodf.shape

(103, 3)

In [14]:
# Delete the first row with postcode as blank
torantodf = torantodf.iloc[1:]
torantodf.head(20)

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
8,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
9,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
10,M1N,Scarborough,"Cliffside West,Birch Cliff"


# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [15]:
unssigned = 'Not assigned'
mvalue = ''
location = 0
for index, row in torantodf.iterrows():
    if(row['Neighborhood'] == unssigned):
        row['Neighborhood'] = row['Borough']

In [16]:
torantodf.head(15)

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
8,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
9,M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside"
10,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [17]:
torantodf.shape

(102, 3)

In [18]:
#Geospatial_Coordinates.csv
# makes the passed rows header 
#df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighborhood"])
newdf = pd.read_csv("Geospatial_Coordinates.csv")
#newdf = pd.read_csv("Geospatial_Coordinates.csv", header = None)


In [19]:
newdf.shape

(103, 3)

In [20]:
newdf.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
#torantodf.head()
originaldf = torantodf.copy()
originaldf.head()

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [22]:
# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [23]:
torantodf.sort_values('Postcode', axis=0, ascending=True, inplace=True)
newdf.sort_values('Postcode', axis=0, ascending=True, inplace=True)

In [24]:
#torantodf = originaldf.copy()

In [25]:
torantodf.head()

,Postcode,Borough,Neighborhood
1,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
3,M1E,Scarborough,"Guildwood,Morningside,West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


# Merge the Torantodf with the "Geospatial_Coordinates.csv" with postcode as the key

In [26]:

# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

torantodf['g'] = torantodf.groupby('Postcode').cumcount()
newdf['g'] = newdf.groupby('Postcode').cumcount()
torantodf.merge(newdf).drop('g',1)
toranto_table = pd.merge(torantodf,newdf,on=["Postcode", 'g'],how='outer').drop('g',1)


In [27]:


# instantiate the dataframe
toranto_table.head()


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [28]:
toranto_table.shape

(103, 5)

In [29]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toranto_table['Borough'].unique()),
        toranto_table.shape[0]
    )
)

The dataframe has 12 boroughs and 103 neighborhoods.


In [30]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [31]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toranto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toranto, Canada city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toranto, Canada city are 43.653963, -79.387207.


In [32]:
# create map of Toranto using latitude and longitude values
map_toranto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toranto_table['Latitude'], toranto_table['Longitude'], toranto_table['Borough'], toranto_table['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toranto)  
    
map_toranto

In [33]:
CLIENT_ID = 'NMOWZ1J0WTQRB5JYTTXGNCD1BUVCMS0Y0TSNMDZVELYTV4WB' # your Foursquare ID
CLIENT_SECRET = '3ALM5M2BNJGMI1XFR0CC32Z3OMFIHT5V4EZ1JQPH2Y4XE13I' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Sunil Kelkar :')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Sunil Kelkar :
CLIENT_ID: NMOWZ1J0WTQRB5JYTTXGNCD1BUVCMS0Y0TSNMDZVELYTV4WB
CLIENT_SECRET:3ALM5M2BNJGMI1XFR0CC32Z3OMFIHT5V4EZ1JQPH2Y4XE13I


In [34]:
toranto_table.loc[0, 'Neighborhood']

'Rouge,Malvern'

In [35]:
neighborhood_latitude = toranto_table.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toranto_table.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toranto_table.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.806686299999996, -79.19435340000001.


In [36]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NMOWZ1J0WTQRB5JYTTXGNCD1BUVCMS0Y0TSNMDZVELYTV4WB&client_secret=3ALM5M2BNJGMI1XFR0CC32Z3OMFIHT5V4EZ1JQPH2Y4XE13I&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [37]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d37e4e36e4650002c441941'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [38]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [40]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


# 2. Explore Neighborhoods in Toranto

In [41]:
#### Let's create a function to repeat the same process to all the neighborhoods in Toranto

In [42]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
toranto_venues = getNearbyVenues(names=toranto_table['Neighborhood'],
                                   latitudes=toranto_table['Latitude'],
                                   longitudes=toranto_table['Longitude']
                                  )


toranto_venues = getNearbyVenues(names=toranto_table['Neighbuorhood'],
                                   latitudes=toranto_table['Latitude'],
                                   longitudes=toranto_table['Longitude']
                                  )

Rouge,Malvern
Port Union,Rouge Hill,Highland Creek
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Golden Mile,Oakridge,Clairlea
Cliffcrest,Scarborough Village West,Cliffside
Cliffside West,Birch Cliff
Wexford Heights,Dorset Park,Scarborough Town Centre
Maryvale,Wexford
Agincourt
Sullivan,Clarks Corners,Tam O'Shanter
Milliken,Agincourt North,L'Amoreaux East,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Oriole,Henry Farm
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South,Flemingdon Park
Wilson Heights,Downsview North,Bathurst Manor
Northwood Park,York University
Downsview East,CFB Toronto
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

KeyError: 'Neighbuorhood'

In [44]:
print(toranto_venues.shape)
toranto_venues.head()

(2243, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge,Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place


Let's check how many venues were returned for each neighborhood

In [45]:

toranto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,South Steeles,Thistletown,Silverstone",9,9,9,9,9,9
Bayview Village,4,4,4,4,4,4
Berczy Park,55,55,55,55,55,55
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
"Cabbagetown,St. James Town",45,45,45,45,45,45
Caledonia-Fairbanks,5,5,5,5,5,5
Canada Post Gateway Processing Centre,11,11,11,11,11,11
Cedarbrae,7,7,7,7,7,7


#### Let's find out how many unique categories can be curated from all the returned venues

In [46]:
print('There are {} uniques categories.'.format(len(toranto_venues['Venue Category'].unique())))

There are 275 uniques categories.


## 3. Analyze Each Neighborhood

In [47]:
# one hot encoding
toranto_onehot = pd.get_dummies(toranto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toranto_onehot['Neighborhood'] = toranto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toranto_onehot.columns[-1]] + list(toranto_onehot.columns[:-1])
toranto_onehot = toranto_onehot[fixed_columns]

toranto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category and then let us confirm the new size

In [48]:
toranto_grouped = toranto_onehot.groupby('Neighborhood').mean().reset_index()
toranto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.0000

In [49]:
toranto_grouped.shape

(97, 275)

### Let's print each neighborhood along with the top 5 most common venues

In [50]:
num_top_venues = 5

for hood in toranto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toranto_grouped[toranto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Sandwich Place  0.25
1    Skating Rink  0.25
2  Breakfast Spot  0.25
3          Lounge  0.25
4   Movie Theater  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,South Steeles,Thistletown,Silverstone----
                 venue  freq
0        Grocery Store  0.22
1          Pizza Place  0.11
2  Fried Chicken Joint  0.11
3           Beer Store  0.11
4          Coffee Shop  0.11


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Music Store  0.00


----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2         Cheese Shop  0.04
3      Farmers Market  0.04
4  Seafood Restaurant  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1          Comic Shop  0.06
2      Farmers Ma

                  venue  freq
0  Fast Food Restaurant  0.12
1    Chinese Restaurant  0.12
2           Pizza Place  0.06
3     Indian Restaurant  0.06
4       Bubble Tea Shop  0.06


----Lawrence Manor East,Bedford Park----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.09
2                 Pub  0.05
3            Pharmacy  0.05
4             Butcher  0.05


----Lawrence Manor,Lawrence Heights----
                    venue  freq
0          Clothing Store  0.23
1  Furniture / Home Store  0.15
2     Arts & Crafts Store  0.08
3                Boutique  0.08
4       Accessories Store  0.08


----Lawrence Park----
                        venue  freq
0  Construction & Landscaping  0.25
1                        Park  0.25
2                 Swim School  0.25
3                    Bus Line  0.25
4                 Yoga Studio  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.12
1     Sporting Goods Shop  0.09
2            Burger J

                             venue  freq
0             Caribbean Restaurant  0.33
1                    Grocery Store  0.33
2                         Bus Line  0.33
3  Molecular Gastronomy Restaurant  0.00
4                           Museum  0.00


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.12
1     Sandwich Place  0.12
2        Yoga Studio  0.06
3               Park  0.06
4        Coffee Shop  0.06


----Toronto Dominion Centre,Design Exchange----
                venue  freq
0         Coffee Shop  0.14
1                Café  0.08
2               Hotel  0.06
3          Restaurant  0.04
4  Italian Restaurant  0.04


----Toronto Islands,Harbourfront East,Union Station----
                venue  freq
0         Coffee Shop  0.11
1            Aquarium  0.05
2               Hotel  0.05
3                Café  0.04
4  Italian Restaurant  0.04


----Trinity,Little Portugal----
               venue  freq
0                Bar  0.11
1        Coffee Shop  0.06
2   As

### Let's put that into a pandas dataframe

In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toranto_grouped['Neighborhood']

for ind in np.arange(toranto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toranto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Sandwich Place,Lounge,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Discount Store,Eastern European Restaurant
1,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Pharmacy,Gift Shop,Electronics Store
2,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Farmers Market,Bakery,Beer Bar,Cheese Shop,Café,Seafood Restaurant,Beach
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Comic Shop,Garden,Brewery,Skate Park,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Butcher


## 4. Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 5 clusters.

In [53]:
# set number of clusters
kclusters = 5

toranto_grouped_clustering = toranto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toranto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int32)

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toranto_grouped['Neighborhood']

for ind in np.arange(toranto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toranto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Sandwich Place,Lounge,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Discount Store,Eastern European Restaurant
1,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Beer Store,Fast Food Restaurant,Pharmacy,Gift Shop,Electronics Store
2,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Women's Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Farmers Market,Bakery,Beer Bar,Cheese Shop,Café,Seafood Restaurant,Beach
4,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Comic Shop,Garden,Brewery,Skate Park,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Butcher


## 4. Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 5 clusters.


In [56]:
# set number of clusters
kclusters = 5

toranto_grouped_clustering = toranto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toranto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1], dtype=int32)

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toranto_merged = toranto_table

# merge toronto_grouped with toronto_table to add latitude/longitude for each neighborhood
toranto_merged = toranto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toranto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Print Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Women's Store
1,M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497,1.0,Moving Target,Bar,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,1.0,Pizza Place,Electronics Store,Intersection,Rental Car Location,Medical Center,Mexican Restaurant,Breakfast Spot,Eastern European Restaurant,Dumpling Restaurant,Diner
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Korean Restaurant,Women's Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Caribbean Restaurant,Thai Restaurant,Bank,Fried Chicken Joint,Dive Bar,Doner Restaurant,Donut Shop


### Finally, let's visualize the resulting clusters


In [61]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toranto_merged['Latitude'], toranto_merged['Longitude'], toranto_merged['Neighborhood'], toranto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        
        fill=True,
        
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters
#color=rainbow[cluster-1],
#fill_color=rainbow[cluster-1],

## 5. Examine Clusters


## Cluster 1

In [62]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 0, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,0.0,Playground,Park,Coffee Shop,Women's Store,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
23,North York,0.0,Park,Bank,Convenience Store,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
25,North York,0.0,Fast Food Restaurant,Food & Drink Shop,Bus Stop,Park,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
30,North York,0.0,Airport,Park,Women's Store,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
31,North York,0.0,Grocery Store,Shopping Mall,Park,Bank,Women's Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,East York,0.0,Park,Pizza Place,Coffee Shop,Convenience Store,Intersection,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
44,Central Toronto,0.0,Park,Bus Line,Construction & Landscaping,Swim School,Women's Store,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
48,Central Toronto,0.0,Playground,Park,Tennis Court,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
50,Downtown Toronto,0.0,Park,Playground,Building,Trail,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant
73,York,0.0,Field,Park,Hockey Arena,Trail,Eastern European Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore


## Cluster 2

In [63]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 1, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,1.0,Moving Target,Bar,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Diner
2,Scarborough,1.0,Pizza Place,Electronics Store,Intersection,Rental Car Location,Medical Center,Mexican Restaurant,Breakfast Spot,Eastern European Restaurant,Dumpling Restaurant,Diner
3,Scarborough,1.0,Coffee Shop,Korean Restaurant,Women's Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,Scarborough,1.0,Hakka Restaurant,Athletics & Sports,Bakery,Caribbean Restaurant,Thai Restaurant,Bank,Fried Chicken Joint,Dive Bar,Doner Restaurant,Donut Shop
6,Scarborough,1.0,Discount Store,Bus Station,Convenience Store,Department Store,Coffee Shop,Chinese Restaurant,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,1.0,Bakery,Bus Line,Fast Food Restaurant,Soccer Field,Bus Station,Drugstore,Dog Run,Doner Restaurant,Donut Shop,Women's Store
8,Scarborough,1.0,Motel,American Restaurant,Women's Store,Diner,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
9,Scarborough,1.0,College Stadium,Skating Rink,General Entertainment,Café,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dim Sum Restaurant
10,Scarborough,1.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant,Drugstore,Dive Bar,Dog Run,Doner Restaurant
11,Scarborough,1.0,Breakfast Spot,Bakery,Smoke Shop,Middle Eastern Restaurant,Women's Store,Dumpling Restaurant,Doner Restaurant,Donut Shop,Drugstore,Electronics Store


## Cluster 3

In [64]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 2, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,2.0,Playground,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Stadium


## Cluster 4

In [65]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 3, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3.0,Fast Food Restaurant,Print Shop,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Women's Store


## Cluster 5

In [66]:
toranto_merged.loc[toranto_merged['Cluster Labels'] == 4, toranto_merged.columns[[1] + list(range(5, toranto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,North York,4.0,Baseball Field,Women's Store,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Discount Store
